In [1]:
import pandas as pd
import os
import logging
from modules.database import init_db, save_to_sql

# Initializing logging for the migration process
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[logging.FileHandler("migration.log"), logging.StreamHandler()]
)

def migrate_excel_to_sql(excel_path):
    """
    Standardizes Excel headers and migrates data to SQLite.
    Includes validation to ensure source file existence and audit logging.
    """
    # 1. Validation: Check if file exists before processing
    if not os.path.exists(excel_path):
        logging.error(f"Source file {excel_path} not found! Migration aborted.")
        return

    logging.info(f"--- Starting Migration from: {excel_path} ---")

    try:
        # 2. Load the data
        df = pd.read_excel(excel_path)
        logging.info(f"Successfully loaded {len(df)} rows from Excel.")
        
        # 3. No Column Mapping! 
        # We keep all 22 columns exactly as they are in Excel 
        # to ensure the Analysis Notebook remains reproducible.
        
        # 4. Data Cleaning for SQL Integrity
        # We only drop rows where the primary key 'Reference' is missing
        if 'Reference' in df.columns:
            df = df.dropna(subset=['Reference'])
        else:
            logging.error("Critical Error: 'Reference' column not found in Excel!")
            return
        
        # Convert the entire DataFrame (all columns) to a list of dicts
        legacy_data = df.to_dict('records')
        
        # 5. Execute Migration
        # Re-initialize the DB with the 22-column schema
        init_db() 
        save_to_sql(legacy_data)
        
        logging.info(f"--- Migration Completed Successfully: {len(legacy_data)} records stored ---")

    except Exception as e:
        # Catching any unexpected errors during Excel reading or Processing
        logging.error(f"Migration failed during processing: {str(e)}")


if __name__ == "__main__":
    # Ensure the path matches your actual file location
    migrate_excel_to_sql('data/raw/ring_details.xlsx')

2026-02-01 00:16:38,873 - INFO - --- Starting Migration from: data/raw/ring_details.xlsx ---
2026-02-01 00:16:39,222 - INFO - Successfully loaded 2303 rows from Excel.
2026-02-01 00:16:39,264 - INFO - --- Migration Completed Successfully: 2303 records stored ---
